# This Notebook presents an expriment of Theory Of Mind on Schelling Coordination Game

In [2]:
# Import libraries.
using Turing, StatsPlots, Random

We have 2 players: Bob and Alice.
They Want to arrive to the same point. they can't communicate, they only can infernce about each other.
We will model it using PPL. And use Practicle Gibbs Sampling.

In [18]:
@model function prior_location(popularity_of_place = 0.9)
    catagorical_dist = zeros(2)
    coin ~ Bernoulli(popularity_of_place)
    catagorical_dist[coin + 1] = 1
    location ~ Categorical(catagorical_dist)
    return location
end

prior_location (generic function with 2 methods)

In [50]:
@model function bob(depth = 0, popularity=0.55)
    my_location ~ Bernoulli(popularity)
    return my_location
end

bob (generic function with 3 methods)

In [53]:
function get_location_2(samples)
    return round(Int64, mean(samples[:my_location]))
end

get_location_2 (generic function with 1 method)

In [54]:
function get_location(samples)
    return samples[:my_location][end]
end

get_location (generic function with 1 method)

In [67]:
@model function alice(other_person_place, popularity = 0.55, depth = 0)
    my_location ~ Bernoulli(popularity)
    other_location = get_location(sample(other_person_place(), PG(10), 10))
    other_location ~ Dirac(my_location)
    return my_location
end

alice (generic function with 3 methods)

In [70]:
samples = sample(alice(bob), PG(10), 10)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (10×4×1 Array{Float64,3}):

Log evidence      = 0.0
Iterations        = 1:10
Thinning interval = 1
Chains            = 1
Samples per chain = 10
parameters        = my_location, other_location
internals         = logevidence, lp

Summary Statistics
      parameters      mean       std   naive_se      mcse       ess      rhat 
          Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

     my_location    0.7000    0.4830     0.1528   missing   10.4762    0.9381
  other_location    0.7000    0.4830     0.1528   missing   10.4762    0.9381

Quantiles
      parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
          Symbol   Float64   Float64   Float64   Float64   Float64 

     my_location    0.0000    0.2500    1.0000    1.0000    1.0000
  other_location    0.0000    0.2500    1.0000    1.0000    1.0000


In [105]:
@model function alice_2(other_person_place, popularity = 0.55, depth = 0)
    my_location ~ Bernoulli(popularity)
    other_location = get_location(sample(other_person_place(alice_2, popularity, depth -1), PG(5), 5))
    other_location ~ Dirac(my_location)
    return my_location
end

alice_2 (generic function with 3 methods)

In [106]:
@model function bob_2(other_person_place, popularity=0.55, depth = 0)
    my_location ~ Bernoulli(popularity)
    if depth == 0
       return my_location 
    else
        other_location = get_location(sample(other_person_place(bob, popularity, depth), PG(5), 5))
        other_location ~ Dirac(my_location)
        return my_location
    end
end

bob_2 (generic function with 3 methods)

In [108]:
samples = sample(alice_2(bob_2, 0.55, 2), PG(5), 5)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:01
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:01
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:01
Sampling: 100%|█████████████████████████████████████████| Time: 

Chains MCMC chain (5×4×1 Array{Float64,3}):

Log evidence      = 0.0
Iterations        = 1:5
Thinning interval = 1
Chains            = 1
Samples per chain = 5
parameters        = my_location, other_location
internals         = logevidence, lp

Summary Statistics
      parameters      mean       std   naive_se      mcse       ess      rhat 
          Symbol   Float64   Float64    Float64   Missing   Float64   Float64 

     my_location    0.0000    0.0000     0.0000   missing       NaN       NaN
  other_location    0.0000    0.0000     0.0000   missing       NaN       NaN

Quantiles
      parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
          Symbol   Float64   Float64   Float64   Float64   Float64 

     my_location    0.0000    0.0000    0.0000    0.0000    0.0000
  other_location    0.0000    0.0000    0.0000    0.0000    0.0000
